## Ссылки
API https://tinkoff.github.io/investAPI/

Библиотка https://github.com/Tinkoff/invest-python

Визуализация https://plotly.com/python/treemaps/

### Как запустить
Нужен токен, который можно сделать на сайте тинькоф

Нужно название счета

In [ ]:
TOKEN = 'tinkoff.api.token'  # можно readonly

account_name = 'Основной счет'

In [ ]:
import time

import pandas as pd
import plotly.express as px
import numpy as np

from tinkoff.invest import Client

In [ ]:



# Просто проверяем что все работает и токен верный
# Получаем список счетов
with Client(TOKEN) as client:
    print(client.users.get_accounts())

In [ ]:
# типы из библиоткеки tinkoff
ID_TYPES = {
    'INSTRUMENT_ID_UNSPECIFIED': 0,
    'INSTRUMENT_ID_TYPE_FIGI': 1,
    'INSTRUMENT_ID_TYPE_TICKER': 2,
    'INSTRUMENT_ID_TYPE_UID': 3,
    'INSTRUMENT_ID_TYPE_POSITION_UID': 4,
}



class MarketMap:
    def __init__(self, token, account_name):
        self.token = token
        self.account_name = account_name

    def get_account(self):
        with Client(self.token) as client:
            accounts = client.users.get_accounts().accounts
            account = [i for i in accounts if i.name == self.account_name]
            assert len(account) == 1
            account = account[0]
            return account
    
    def get_account_portfolio(self, update=False):
        account = self.get_account()
        
        with Client(self.token) as client:
            portfolio = client.operations.get_portfolio(account_id=account.id)
            return portfolio
    
    def get_info_by_figi(self, portfolio):
        instruments_info = {}
        with Client(self.token) as client:
            for pos in portfolio.positions:
                #print(pos.figi, pos.instrument_type, pos.quantity, pos.current_price)
                #if pos.instrument_type in ['bond', 'etf', 'futures']:
                    #continue
                if pos.instrument_type == 'currency':
                    info = client.instruments.currency_by(id_type=1, id=pos.figi)
                elif pos.instrument_type == 'etf':
                    info = client.instruments.etf_by(id_type=1, id=pos.figi)
                elif pos.instrument_type == 'bond':
                    info = client.instruments.bond_by(id_type=1, id=pos.figi)
                elif pos.instrument_type == 'futures':
                    info = client.instruments.future_by(id_type=1, id=pos.figi)
                else:
                    info = client.instruments.share_by(id_type=1, id=pos.figi)
                instruments_info[pos.figi] = info
                time.sleep(0.3)               

            return instruments_info

    
def to_int(value):
    return value.units + (value.nano / 1000000000 if value.nano else 0)


def get_portfolio_value(portfolio):
    total_amount = [
        portfolio.total_amount_shares,
        portfolio.total_amount_bonds,
        portfolio.total_amount_etf,
        portfolio.total_amount_currencies,
        #portfolio.total_amount_futures,
    ]
    total_amount = sum(map(to_int, total_amount))
    
    return total_amount


def get_positions_by_type(portfolio):
    types = ['currency', 'share', 'bond', 'etf', 'futures']
    positions_by_type = {}
    for pos_type in types:
        positions_by_type[pos_type] = [i for i in portfolio.positions if i.instrument_type == pos_type]
    return positions_by_type

def get_currency_prices(portfolio, instruments_info):
    cur_pos = get_positions_by_type(portfolio)['currency']
    currencies_current_price = {}
    
    for c_pos in cur_pos:
        current_price = to_int(c_pos.current_price)
        currency_name = instruments_info[c_pos.figi].instrument.iso_currency_name
        currencies_current_price[currency_name] = current_price
    return currencies_current_price

def get_shares_share(portfolio, instruments_info):
    shares_share = []
    currencies_current_price = get_currency_prices(portfolio, instruments_info)
    portfolio_value = get_portfolio_value(portfolio)
    assert portfolio_value > 0
    
    share_pos = get_positions_by_type(portfolio)['share']
    for s_pos in share_pos:
        _info = instruments_info[s_pos.figi].instrument
        currency = s_pos.current_price.currency
        if currency == 'rub':
            position_value_rub = to_int(s_pos.current_price) * to_int(s_pos.quantity)
        else:
            position_value_rub = to_int(s_pos.current_price) * currencies_current_price[currency] * to_int(s_pos.quantity)
        shares_share.append({
            'ticker': _info.ticker,
            'position_value_rub': position_value_rub,
            'quantity': to_int(s_pos.quantity),
            'position_share': position_value_rub / portfolio_value,
            'currency': currency,
        })
    return shares_share


# добавляем данные по облигациям, ETF и валютам
def update_shares_share(portfolio, shares_share):
    portfolio_value = get_portfolio_value(portfolio)

    shares_share.append({
        'ticker': 'bonds',
        'position_value_rub': to_int(portfolio.total_amount_bonds),
        'quantity': 1,
        'position_share': to_int(portfolio.total_amount_bonds) / portfolio_value,
        'currency': 'bonds, etf and currencies',
    })
    shares_share.append({
        'ticker': 'etf',
        'position_value_rub': to_int(portfolio.total_amount_etf),
        'quantity': 1,
        'position_share': to_int(portfolio.total_amount_etf) / portfolio_value,
        'currency': 'bonds, etf and currencies',
    })
    shares_share.append({
        'ticker': 'currencies',
        'position_value_rub': to_int(portfolio.total_amount_currencies),
        'quantity': 1,
        'position_share': to_int(portfolio.total_amount_currencies) / portfolio_value,
        'currency': 'bonds, etf and currencies',
    })
    
    return shares_share
    
    

In [ ]:
# Подставляем токен и название нужного счета
mmap = MarketMap(TOKEN, account_name)

In [ ]:
portfolio = mmap.get_account_portfolio()

In [ ]:
# средств на счете
get_portfolio_value(portfolio)

In [ ]:
instruments_info = mmap.get_info_by_figi(portfolio)

In [ ]:
# получаем курс валют (все нужные мне есть у меня в портфеле
# если у вас их нет на остатке, то нужно будет придумать другой подход получить курс)
currencies_current_price = get_currency_prices(portfolio, instruments_info)
currencies_current_price

In [ ]:
# считаем доли для всех акций
shares_share = get_shares_share(portfolio, instruments_info)
shares_share

In [ ]:
# добавляем инфу по ETF, облигациям и валютам
shares_share = update_shares_share(portfolio, shares_share)

In [ ]:
shares_share_df = pd.DataFrame(shares_share)
shares_share_df

In [ ]:
# Строим график

fig = px.treemap(shares_share_df, path=[px.Constant('portfolio'), 'currency', 'ticker'], values='position_value_rub',
                  color='position_value_rub',# hover_data=['iso_alpha'],
                  color_continuous_scale='RdBu',
                  #color_continuous_midpoint=np.average(df['lifeExp'], weights=df['pop'])
)
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

### Шаблон для графика взят отсюда
https://plotly.com/python/treemaps/

In [ ]:
df = px.data.gapminder().query("year == 2007")
fig = px.treemap(df, path=[px.Constant("world"), 'continent', 'country'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df['lifeExp'], weights=df['pop']))
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [ ]:
df